Directional Trading Strategy

Downtrend (Underlying's price < 50 day SMA)
 1. If %K > 60 (overbought) and %K drops 5% then:
 2. Buy a >= 45 DTE 1 or 2 ITM Put with OI >= 100 and priced at bid-ask mid-point*
 3. Exit the position, sell the Put, before expiration when:
    a. Put price drops 50%
    b. Underlying's price > previous day's High & %K < 40 (i.e. is oversold)
    c. Sell half of position if Put price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

Uptrend (Underlying's price > 50 day SMA)
1. If %K < 40 (oversold) and %K gains 5% then:
2. Buy a >= 45 DTE 1 or 2 ITM Call with OI >= 100 and priced at bid-ask mid-point*
3. Exit the position, sell the Call, before expiration when:
    a. Call price drops 50%
    b. Underlying's price < previous day's Low & %K > 60 (overbought)
    c. Sell half of position if Call price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

    *Risk Management: Option Price * 100 <= 5% of trading capital

In [8]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
import pandas as pd
import os

# Load datasets
equity_price_series = pd.read_csv('ge.stock.20080101.1231.csv', index_col='date')

In [9]:
equity_price_series.head()

,open,high,low,last,volume,option volume,implied vol
date,,,,,,,
2008-01-02,37.10,37.45,36.55,36.76,38799202,60299,0.258324
2008-01-03,36.81,37.22,36.70,36.80,26800382,67025,0.250877
2008-01-04,36.54,36.67,35.98,36.04,40729626,83956,0.276142
2008-01-07,36.21,36.50,35.91,36.18,47509039,53254,0.293888
2008-01-08,36.43,36.43,35.20,35.40,44824259,75982,0.316222


In [39]:
highs = equity_price_series.iloc[:, 1]
lows = equity_price_series.iloc[:, 2]
close = equity_price_series.iloc[:, 3]

H14 = highs.rolling(14).max()
L14 = lows.rolling(14).min()
equity_price_series['K'] = 100*(close-L14)/(H14-L14)

In [51]:
equity_price_series['mid'] = (equity_price_series.iloc[:,1]-equity_price_series.iloc[:,2])/2+equity_price_series.iloc[:,2]

In [53]:
equity_price_series['50sma'] = equity_price_series['mid'].rolling(50).mean()

In [54]:
equity_price_series

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma
date,,,,,,,,,,
2008-01-02,37.10,37.45,36.55,36.76,38799202,60299,0.258324,NaN,37.000,NaN
2008-01-03,36.81,37.22,36.70,36.80,26800382,67025,0.250877,NaN,36.960,NaN
2008-01-04,36.54,36.67,35.98,36.04,40729626,83956,0.276142,NaN,36.325,NaN
2008-01-07,36.21,36.50,35.91,36.18,47509039,53254,0.293888,NaN,36.205,NaN
2008-01-08,36.43,36.43,35.20,35.40,44824259,75982,0.316222,NaN,35.815,NaN
...,...,...,...,...,...,...,...,...,...,...
2008-12-24,16.10,16.23,15.92,16.11,23409169,27415,0.537966,10.140845,16.075,17.5699
2008-12-26,16.05,16.13,15.78,15.97,29654708,46953,0.554954,6.197183,15.955,17.4919
2008-12-29,15.95,16.00,15.35,15.66,55804550,70844,0.606879,9.365559,15.675,17.4227


In [58]:
i = 0
equity_price_series['trend'] = None

while i < len(equity_price_series):
    if equity_price_series.iloc[i, 8] > equity_price_series.iloc[i, 9]:
        equity_price_series.iloc[i, 10] = 'up'
    elif equity_price_series.iloc[i, 8] < equity_price_series.iloc[i, 9]:
        equity_price_series.iloc[i, 10] = 'down'
    i = i + 1

In [60]:
equity_price_series.tail()

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend
date,,,,,,,,,,,
2008-12-24,16.10,16.23,15.92,16.11,23409169,27415,0.537966,10.140845,16.075,17.5699,down
2008-12-26,16.05,16.13,15.78,15.97,29654708,46953,0.554954,6.197183,15.955,17.4919,down
2008-12-29,15.95,16.00,15.35,15.66,55804550,70844,0.606879,9.365559,15.675,17.4227,down
2008-12-30,15.80,15.85,15.55,15.82,57449351,99902,0.590034,15.161290,15.700,17.3402,down
2008-12-31,15.82,16.34,15.77,16.20,60224914,78096,0.524874,30.575540,16.055,17.2633,down


In [64]:
i = 0
equity_price_series['option2buy'] = None

while i < len(equity_price_series):
    if equity_price_series.iloc[i, 10] == 'up': 
        if equity_price_series.iloc[i, 7] <= 40:
            equity_price_series.iloc[i, 11] = 'call'
    elif equity_price_series.iloc[i, 10] == 'down': 
        if equity_price_series.iloc[i, 7] >= 60:
            equity_price_series.iloc[i, 11] = 'put'
    i = i + 1

In [70]:
equity_price_series[48:100]

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend,option2buy
date,,,,,,,,,,,,
2008-03-12,33.45,34.46,33.45,33.96,70642944,120363,0.264458,82.206406,33.955,NaN,None,None
2008-03-13,33.51,34.51,33.31,34.25,64523914,150731,0.263034,90.909091,33.910,34.4604,down,put
2008-03-14,34.52,34.57,33.06,33.82,73097516,143860,0.336645,74.315068,33.815,34.3967,down,put
2008-03-17,32.99,34.73,32.83,34.33,80415892,88161,0.345919,87.012987,33.780,34.3331,down,put
2008-03-18,34.91,36.22,34.71,36.14,83668824,184883,0.328818,98.249453,35.465,34.3159,up,None
2008-03-19,36.20,36.63,35.55,35.59,64817457,106356,0.289941,79.116466,36.090,34.3136,up,None
2008-03-20,36.54,37.74,36.40,37.49,110630359,193933,0.281720,95.894910,37.070,34.3387,up,None
2008-03-24,37.63,37.67,37.04,37.40,46992147,130230,0.276616,94.417077,37.355,34.3749,up,None
2008-03-25,37.25,37.53,36.92,37.27,44998693,89819,0.265928,92.282430,37.225,34.4034,up,None


In [88]:
i = 0
j = 0

equity_price_series['K_diff'] = None

while i < len(equity_price_series):
    if i >= 14:
        equity_price_series.iloc[i, 13] = equity_price_series.iloc[i, 7] - equity_price_series.iloc[j, 7]
    i = i + 1
    j = i - 1

In [90]:
i = 0
equity_price_series['signal'] = None

while i < len(equity_price_series):
    if equity_price_series.iloc[i, 11] == 'put': 
        if equity_price_series.iloc[i, 13] <= -5:
            equity_price_series.iloc[i, 12] = 'buy_put'
    elif equity_price_series.iloc[i, 11] == 'call': 
        if equity_price_series.iloc[i, 13] >= 5:
            equity_price_series.iloc[i, 12] = 'buy_call'
    i = i + 1

In [95]:
equity_price_series[48:100]

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend,option2buy,signal,K_diff
date,,,,,,,,,,,,,,
2008-03-12,33.45,34.46,33.45,33.96,70642944,120363,0.264458,82.206406,33.955,NaN,None,None,None,17.63076
2008-03-13,33.51,34.51,33.31,34.25,64523914,150731,0.263034,90.909091,33.910,34.4604,down,put,None,8.702685
2008-03-14,34.52,34.57,33.06,33.82,73097516,143860,0.336645,74.315068,33.815,34.3967,down,put,buy_put,-16.594022
2008-03-17,32.99,34.73,32.83,34.33,80415892,88161,0.345919,87.012987,33.780,34.3331,down,put,None,12.697919
2008-03-18,34.91,36.22,34.71,36.14,83668824,184883,0.328818,98.249453,35.465,34.3159,up,None,None,11.236466
2008-03-19,36.20,36.63,35.55,35.59,64817457,106356,0.289941,79.116466,36.090,34.3136,up,None,None,-19.132987
2008-03-20,36.54,37.74,36.40,37.49,110630359,193933,0.281720,95.894910,37.070,34.3387,up,None,None,16.778444
2008-03-24,37.63,37.67,37.04,37.40,46992147,130230,0.276616,94.417077,37.355,34.3749,up,None,None,-1.477833
2008-03-25,37.25,37.53,36.92,37.27,44998693,89819,0.265928,92.282430,37.225,34.4034,up,None,None,-2.134647


In [97]:
equity_price_series.to_csv('ge.option_trading_signals.2008.csv')

Conditions to enter and exit positions:

Downtrend (Underlying's price < 50 day SMA)
 1. If %K > 60 (overbought) and %K drops 5% then:
 2. Buy a >= 45 DTE 1 or 2 ITM Put with OI >= 100 and priced at bid-ask mid-point*
 3. Exit the position, sell the Put, before expiration when:
    a. Put price drops 50%
    b. Underlying's price > previous day's High & %K < 40 (i.e. is oversold)
    c. Sell half of position if Put price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

Uptrend (Underlying's price > 50 day SMA)
1. If %K < 40 (oversold) and %K gains 5% then:
2. Buy a >= 45 DTE 1 or 2 ITM Call with OI >= 100 and priced at bid-ask mid-point*
3. Exit the position, sell the Call, before expiration when:
    a. Call price drops 50%
    b. Underlying's price < previous day's Low & %K > 60 (overbought)
    c. Sell half of position if Call price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

    *Risk Management: Option Price * 100 <= 5% of trading capital

In [35]:
import pandas as pd

options_chain = pd.read_csv('ge.options.20080101.1231.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
equity_price_series = pd.read_csv('ge.option_trading_signals.2008.csv', index_col=0, parse_dates=True, infer_datetime_format=True)

In [36]:
from datetime import datetime, date, time, timedelta

options_chain['expiration_dates'] = None

i = 0

for expiry in options_chain.iloc[:, 2]:
    options_chain.iloc[i, 17] = datetime.strptime(expiry, '%Y-%m-%d')
    i = i + 1

In [37]:
options_chain.tail()

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-12-31,VGEMW,GE,2011-01-22,17.5,P,5.60,5.80,5.85,77,313,0.490363,-0.404404,0.034002,-0.244031,-0.002939,0.090032,0,2011-01-22 00:00:00
2008-12-31,WGEAC,GE,2010-01-16,15.0,C,3.50,3.70,3.61,1148,14411,0.536153,0.660653,0.041315,0.069831,-0.004270,0.060522,0,2010-01-16 00:00:00
2008-12-31,WGEAW,GE,2010-01-16,17.5,C,2.37,2.51,2.38,232,22068,0.496936,0.540352,0.048320,0.062395,-0.004290,0.065606,0,2010-01-16 00:00:00
2008-12-31,WGEMC,GE,2010-01-16,15.0,P,3.00,3.10,3.10,2402,42230,0.542861,-0.338740,0.040776,-0.086652,-0.004320,0.060480,0,2010-01-16 00:00:00
2008-12-31,WGEMW,GE,2010-01-16,17.5,P,4.25,4.40,4.30,170,32133,0.493023,-0.460930,0.048719,-0.119744,-0.004257,0.065627,0,2010-01-16 00:00:00


In [4]:
# todo: select contracts from options chain that match the buy signal

In [38]:
signals = equity_price_series.loc[(equity_price_series['signal']=='buy_put')|(equity_price_series['signal']=='buy_call')]

In [39]:
signals

,open,high,low,last,volume,option volume,implied vol,K,mid,50sma,trend,option2buy,signal,K_diff
date,,,,,,,,,,,,,,
2008-03-14,34.52,34.57,33.06,33.82,73097516,143860,0.336645,74.315068,33.815,34.3967,down,put,buy_put,-16.594022
2008-04-10,36.40,37.07,36.16,36.75,44175033,124576,0.330186,25.000000,36.615,35.0452,up,call,buy_call,15.043290
2008-05-05,33.36,33.36,32.96,33.18,26460612,54290,0.216213,71.084337,33.160,34.4187,down,put,buy_put,-13.749786
2008-05-06,33.07,33.17,32.62,33.00,41743801,104875,0.237848,60.240964,32.895,34.3984,down,put,buy_put,-10.843373
2008-07-21,28.12,28.18,27.61,27.69,40998771,90848,0.263080,74.113475,27.895,29.2826,down,put,buy_put,-10.992908
2008-07-31,28.70,28.96,28.29,28.29,54538295,43085,0.290855,62.703963,28.625,28.6922,down,put,buy_put,-15.850816
2008-08-28,28.35,28.91,28.30,28.83,42528267,65976,0.279782,35.269710,28.605,28.2032,up,call,buy_call,25.311203
2008-09-02,28.54,29.15,28.44,28.53,57843774,45675,0.294318,23.605150,28.795,28.2355,up,call,buy_call,18.454936
2008-11-05,20.69,20.98,19.80,19.93,86648062,164546,0.607745,70.557029,20.390,23.2378,down,put,buy_put,-22.281167


In [6]:
# filter options contracts for buy signal
i = 0

options = pd.DataFrame()
dates = signals.index

while i < len(signals):
    options = options.append(options_chain.loc[(options_chain.index==dates[i])])
    i = i + 1

In [7]:
options.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1382 entries, 2008-03-14 to 2008-12-11
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0    symbol           1382 non-null   object 
 1    under            1382 non-null   object 
 2    expiration       1382 non-null   object 
 3    strike           1382 non-null   float64
 4    put/call         1382 non-null   object 
 5    bid              1382 non-null   float64
 6    ask              1382 non-null   float64
 7    price            1382 non-null   float64
 8    volume           1382 non-null   int64  
 9    open interest    1382 non-null   int64  
 10   implied vol      1283 non-null   float64
 11   delta            1382 non-null   float64
 12   gamma            1382 non-null   float64
 13   rho              1382 non-null   float64
 14   theta            1382 non-null   float64
 15   vega             1382 non-null   float64
 16   nonstd           1382 n

In [8]:
options

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-03-14,GECC,GE,2008-03-22,31.0,C,2.90,2.96,2.93,297,6428,0.497979,0.902681,0.073764,0.005293,-0.028662,0.008058,0,2008-03-22 00:00:00
2008-03-14,GECF,GE,2008-03-22,30.0,C,3.85,4.00,4.00,15,8159,0.625794,0.923114,0.049219,0.005235,-0.030201,0.006756,0,2008-03-22 00:00:00
2008-03-14,GECG,GE,2008-03-22,35.0,C,0.19,0.20,0.20,6349,61878,0.322267,0.227791,0.200090,0.001440,-0.032560,0.014145,0,2008-03-22 00:00:00
2008-03-14,GECN,GE,2008-03-22,34.0,C,0.52,0.56,0.54,8099,24180,0.334112,0.463514,0.253875,0.002903,-0.044405,0.018607,0,2008-03-22 00:00:00
2008-03-14,GECO,GE,2008-03-22,36.0,C,0.06,0.07,0.07,913,11057,0.332842,0.091404,0.105374,0.000580,-0.018291,0.007694,0,2008-03-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-11,WGEAD,GE,2010-01-16,20.0,C,2.69,2.78,2.78,171,52969,0.595192,0.522113,0.037496,0.062497,-0.005290,0.071098,0,2010-01-16 00:00:00
2008-12-11,WGEAW,GE,2010-01-16,17.5,C,3.75,3.90,3.80,208,20151,0.650153,0.618701,0.032846,0.067604,-0.005529,0.068032,0,2010-01-16 00:00:00
2008-12-11,WGEMC,GE,2010-01-16,15.0,P,3.65,3.80,3.80,1059,39371,0.670090,-0.296891,0.028933,-0.093165,-0.005173,0.061766,0,2010-01-16 00:00:00


In [58]:
# filter options contracts for approx. 45 DTE
delta = timedelta(days=45)

contracts_dte = pd.DataFrame()

i = 0

while i < len(options):
    dte = options.index[i] + delta
    contracts_dte = contracts_dte.append(options[options['expiration_dates']==dte])
    i = i + 1

In [59]:
contracts_dte

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-05-05,GELA,GE,2008-12-20,28.0,C,5.70,5.85,5.75,12,140,0.310317,0.792529,0.035172,0.125158,-0.005118,0.075058,0,2008-12-20 00:00:00
2008-05-05,GELB,GE,2008-12-20,29.0,C,4.95,5.05,4.95,82,131,0.297230,0.755260,0.040316,0.122425,-0.005381,0.082408,0,2008-12-20 00:00:00
2008-05-05,GELC,GE,2008-12-20,31.0,C,3.50,3.60,3.59,11,50,0.269741,0.664891,0.051516,0.113117,-0.005661,0.095562,0,2008-12-20 00:00:00
2008-05-05,GELF,GE,2008-12-20,30.0,C,4.20,4.30,4.45,0,227,0.282957,0.713217,0.045896,0.118567,-0.005550,0.089309,0,2008-12-20 00:00:00
2008-05-05,GELG,GE,2008-12-20,35.0,C,1.48,1.53,1.52,20,1863,0.238924,0.425475,0.062562,0.077210,-0.005390,0.102793,0,2008-12-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-11,GEWXK,GE,2008-12-20,17.5,P,1.14,1.17,1.14,11082,77343,0.894949,-0.551863,0.175105,-0.002315,-0.055850,0.009985,0,2008-12-20 00:00:00
2008-12-11,GEWXN,GE,2008-12-20,14.0,P,0.07,0.10,0.08,1939,16699,0.966449,-0.073662,0.057234,-0.000294,-0.021288,0.003524,0,2008-12-20 00:00:00
2008-12-11,GEWXP,GE,2008-12-20,19.0,P,2.33,2.38,2.30,1503,11023,1.054370,-0.730950,0.124017,-0.003248,-0.054902,0.008331,0,2008-12-20 00:00:00


In [60]:
calls = contracts_dte[contracts_dte.iloc[:, 4]=='C']

In [61]:
calls

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-05-05,GELA,GE,2008-12-20,28.0,C,5.70,5.85,5.75,12,140,0.310317,0.792529,0.035172,0.125158,-0.005118,0.075058,0,2008-12-20 00:00:00
2008-05-05,GELB,GE,2008-12-20,29.0,C,4.95,5.05,4.95,82,131,0.297230,0.755260,0.040316,0.122425,-0.005381,0.082408,0,2008-12-20 00:00:00
2008-05-05,GELC,GE,2008-12-20,31.0,C,3.50,3.60,3.59,11,50,0.269741,0.664891,0.051516,0.113117,-0.005661,0.095562,0,2008-12-20 00:00:00
2008-05-05,GELF,GE,2008-12-20,30.0,C,4.20,4.30,4.45,0,227,0.282957,0.713217,0.045896,0.118567,-0.005550,0.089309,0,2008-12-20 00:00:00
2008-05-05,GELG,GE,2008-12-20,35.0,C,1.48,1.53,1.52,20,1863,0.238924,0.425475,0.062562,0.077210,-0.005390,0.102793,0,2008-12-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-11,GEWLK,GE,2008-12-20,17.5,C,0.49,0.51,0.50,12704,97028,0.688543,0.418946,0.224785,0.001456,-0.042438,0.009862,0,2008-12-20 00:00:00
2008-12-11,GEWLN,GE,2008-12-20,14.0,C,3.05,3.15,3.15,263,9361,1.033130,0.913884,0.060255,0.002723,-0.025611,0.003966,0,2008-12-20 00:00:00
2008-12-11,GEWLP,GE,2008-12-20,19.0,C,0.14,0.15,0.15,8802,50439,0.704163,0.161913,0.137954,0.000573,-0.027240,0.006189,0,2008-12-20 00:00:00


In [62]:
puts = contracts_dte[contracts_dte.iloc[:, 4]=='P']

In [63]:
puts

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-05-05,GEXA,GE,2008-12-20,28.0,P,0.70,0.72,0.72,0,1257,0.242642,-0.163211,0.038738,-0.037582,-0.003439,0.064639,0,2008-12-20 00:00:00
2008-05-05,GEXB,GE,2008-12-20,29.0,P,0.90,0.93,0.92,38,337,0.233901,-0.205779,0.046427,-0.047511,-0.003830,0.074679,0,2008-12-20 00:00:00
2008-05-05,GEXC,GE,2008-12-20,31.0,P,1.46,1.49,1.46,67,4904,0.216588,-0.314561,0.062510,-0.073116,-0.004420,0.093106,0,2008-12-20 00:00:00
2008-05-05,GEXF,GE,2008-12-20,30.0,P,1.15,1.18,1.16,28,7801,0.224947,-0.255933,0.054541,-0.059265,-0.004160,0.084371,0,2008-12-20 00:00:00
2008-05-05,GEXG,GE,2008-12-20,35.0,P,3.35,3.45,3.40,0,1821,0.182671,-0.616882,0.079689,-0.146612,-0.004003,0.100106,0,2008-12-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-11,GEWXK,GE,2008-12-20,17.5,P,1.14,1.17,1.14,11082,77343,0.894949,-0.551863,0.175105,-0.002315,-0.055850,0.009985,0,2008-12-20 00:00:00
2008-12-11,GEWXN,GE,2008-12-20,14.0,P,0.07,0.10,0.08,1939,16699,0.966449,-0.073662,0.057234,-0.000294,-0.021288,0.003524,0,2008-12-20 00:00:00
2008-12-11,GEWXP,GE,2008-12-20,19.0,P,2.33,2.38,2.30,1503,11023,1.054370,-0.730950,0.124017,-0.003248,-0.054902,0.008331,0,2008-12-20 00:00:00


In [64]:
calls = calls[signals.iloc[:, 11]=='call']

<ipython-input-64-a0499abd928d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  calls = calls[signals.iloc[:, 11]=='call']


In [65]:
calls

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-08-28,DOCLA,GE,2008-12-20,27.0,C,0.53,0.73,0.00,0,0,0.162617,0.779435,0.113681,0.062831,-0.003427,0.047570,102,2008-12-20 00:00:00
2008-08-28,DOCLB,GE,2008-12-20,28.0,C,0.50,0.70,0.75,0,0,0.103107,0.704796,0.208686,0.059352,-0.002530,0.055368,102,2008-12-20 00:00:00
2008-08-28,DOCLC,GE,2008-12-20,29.0,C,0.38,0.58,0.00,0,0,0.108654,0.473416,0.228387,0.040348,-0.003072,0.063854,102,2008-12-20 00:00:00
2008-08-28,DOCLD,GE,2008-12-20,30.0,C,0.26,0.46,0.00,0,0,0.141078,0.320151,0.158054,0.027176,-0.003583,0.057377,102,2008-12-20 00:00:00
2008-08-28,DOCLE,GE,2008-12-20,31.0,C,0.15,0.35,0.00,0,0,0.160371,0.221112,0.115432,0.018766,-0.003381,0.047635,102,2008-12-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-09-02,GEWLC,GE,2008-12-20,29.0,C,1.43,1.47,1.47,186,11262,0.292367,0.490786,0.087903,0.036703,-0.008380,0.061897,0,2008-12-20 00:00:00
2008-09-02,GEWLD,GE,2008-12-20,30.0,C,1.01,1.05,1.01,336,23220,0.282146,0.401104,0.088298,0.030457,-0.007839,0.060001,0,2008-12-20 00:00:00
2008-09-02,GEWLX,GE,2008-12-20,24.0,C,4.80,4.95,4.70,0,753,0.377879,0.827436,0.043569,0.054672,-0.006941,0.039652,0,2008-12-20 00:00:00


In [66]:
signals.loc['2008-08-28']

 open                 28.35
 high                 28.91
 low                   28.3
 last                 28.83
 volume            42528267
 option volume        65976
 implied vol       0.279782
K                  35.26971
mid                  28.605
50sma               28.2032
trend                    up
option2buy             call
signal             buy_call
K_diff            25.311203
Name: 2008-08-28 00:00:00, dtype: object

In [79]:
itm_calls = calls[(calls.index=='2008-08-28') & (calls[' open interest']>800) & (calls[' strike']>=28.5) & (calls[' strike']<=29.5) & (calls[' price']<=2.50)]

In [80]:
itm_calls

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00
2008-08-28,GEWLC,GE,2008-12-20,29.0,C,1.58,1.61,1.6,184,11255,0.286137,0.517078,0.086838,0.04073,-0.008098,0.063938,0,2008-12-20 00:00:00


In [81]:
puts = contracts_dte[contracts_dte.iloc[:, 4]=='P']
puts = puts[signals.iloc[:, 11]=='put']

<ipython-input-81-93a4bb86e4b9>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  puts = puts[signals.iloc[:, 11]=='put']


In [82]:
puts

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-05-05,GEXA,GE,2008-12-20,28.0,P,0.70,0.72,0.72,0,1257,0.242642,-0.163211,0.038738,-0.037582,-0.003439,0.064639,0,2008-12-20 00:00:00
2008-05-05,GEXB,GE,2008-12-20,29.0,P,0.90,0.93,0.92,38,337,0.233901,-0.205779,0.046427,-0.047511,-0.003830,0.074679,0,2008-12-20 00:00:00
2008-05-05,GEXC,GE,2008-12-20,31.0,P,1.46,1.49,1.46,67,4904,0.216588,-0.314561,0.062510,-0.073116,-0.004420,0.093106,0,2008-12-20 00:00:00
2008-05-05,GEXF,GE,2008-12-20,30.0,P,1.15,1.18,1.16,28,7801,0.224947,-0.255933,0.054541,-0.059265,-0.004160,0.084371,0,2008-12-20 00:00:00
2008-05-05,GEXG,GE,2008-12-20,35.0,P,3.35,3.45,3.40,0,1821,0.182671,-0.616882,0.079689,-0.146612,-0.004003,0.100106,0,2008-12-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-11,GEWXK,GE,2008-12-20,17.5,P,1.14,1.17,1.14,11082,77343,0.894949,-0.551863,0.175105,-0.002315,-0.055850,0.009985,0,2008-12-20 00:00:00
2008-12-11,GEWXN,GE,2008-12-20,14.0,P,0.07,0.10,0.08,1939,16699,0.966449,-0.073662,0.057234,-0.000294,-0.021288,0.003524,0,2008-12-20 00:00:00
2008-12-11,GEWXP,GE,2008-12-20,19.0,P,2.33,2.38,2.30,1503,11023,1.054370,-0.730950,0.124017,-0.003248,-0.054902,0.008331,0,2008-12-20 00:00:00


In [83]:
signals.loc['2008-05-05']

 open                 33.36
 high                 33.36
 low                  32.96
 last                 33.18
 volume            26460612
 option volume        54290
 implied vol       0.216213
K                 71.084337
mid                   33.16
50sma               34.4187
trend                  down
option2buy              put
signal              buy_put
K_diff           -13.749786
Name: 2008-05-05 00:00:00, dtype: object

In [84]:
itm_puts = puts[(puts.index=='2008-05-05') & (puts[' open interest']>800) & (puts[' strike']<=33.5) & (puts[' strike']>=32.5) & (puts[' price'] < 2.50)]

In [85]:
itm_puts

,symbol,under,expiration,strike,put/call,bid,ask,price,volume,open interest,implied vol,delta,gamma,rho,theta,vega,nonstd,expiration_dates
date,,,,,,,,,,,,,,,,,,
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00
2008-05-05,GEXU,GE,2008-12-20,33.0,P,2.27,2.32,2.32,10,5287,0.200385,-0.454653,0.075432,-0.106717,-0.004563,0.103948,0,2008-12-20 00:00:00


In [ ]:
# todo: sell options contract when exit criteria met:
'''
Downtrend (Underlying's price < 50 day SMA)
 Exit the position, sell the Put, before expiration when:
    a. Put price drops 50%
    b. Underlying's price > previous day's High & %K < 40 (i.e. is oversold)
    c. Sell half of position if Put price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

Uptrend (Underlying's price > 50 day SMA)
 Exit the position, sell the Call, before expiration when:
    a. Call price drops 50%
    b. Underlying's price < previous day's Low & %K > 60 (overbought)
    c. Sell half of position if Call price gains 80%
    d. Sell at time T (maximum holding period with no gain or loss)

    *Risk Management: Option Price * 100 <= 5% of trading capital
'''